In [1]:
import numpy as np
np.set_printoptions(precision=3, suppress=True) # Round the results to 3 decimal place

In [2]:
def householderReflection(x_hut, num, num_rc):
    I = np.identity(num_rc)
    x = np.vstack((np.zeros((num,1)), x_hut.reshape(-1,1))) # append zero above the target
    # change the operation depending on the sign
    if np.sign(x[num]) == 0 or np.sign(x[num]) == 1:
        x[num] += np.linalg.norm(x)
    else:
        x[num] -= np.linalg.norm(x)
    u = x/np.linalg.norm(x) # normalize
    H = I - 2*np.outer(u, u) # householder reflection
    return H

In [3]:
def golubKahanBidiagonalization(matrix):
    A = matrix.astype('float')
    num_rows, num_columns = matrix.shape
    U = np.identity(num_rows)
    V_T = np.identity(num_columns)
    B = A.copy() # initial bidiagonal matrix
    row = 0
    column = 0
    while row<num_rows-1 or column<num_columns-2:
        if row<num_rows-1:
            U_hut = householderReflection(B[row:, row], row, num_rows)
            B = U_hut @ B
            row+=1
            U = U @ U_hut
        if column<num_columns-2:
            V_hut = householderReflection(B[column, column+1:], column+1, num_columns)
            B = B @ V_hut
            column+=1
            V_T = V_hut @ V_T
    return U, B, V_T

In [4]:
# matrix = np.array([[1, 0, 2, 3], [-1, 0, 5, 2], [2, -2, 0, 0], [2, -1, 2, 0]]) # sample matrix
matrix = np.random.randint(0, 10, size=(4, 5))
print('Original Matrix\n', matrix)

Original Matrix
 [[4 7 9 6 9]
 [3 9 3 3 8]
 [2 4 5 9 3]
 [1 8 8 4 6]]


In [5]:
U, B, V_T = golubKahanBidiagonalization(matrix)
print('U Matrix\n', U)
print('Bidiagonal Matrix\n', B)
print('V_T Matrix\n', V_T)

U Matrix
 [[-0.73  -0.212  0.528  0.378]
 [-0.548 -0.128 -0.827  0.024]
 [-0.365  0.135  0.194 -0.9  ]
 [-0.183  0.959 -0.021  0.214]]
Bidiagonal Matrix
 [[-5.477 23.958  0.    -0.     0.   ]
 [ 0.    -9.134 -2.562 -0.     0.   ]
 [-0.    -0.    -1.808 -4.85   0.   ]
 [-0.    -0.     0.     3.207  4.467]]
V_T Matrix
 [[ 1.     0.     0.     0.     0.   ]
 [ 0.    -0.541 -0.48  -0.419 -0.549]
 [ 0.    -0.251 -0.655  0.167  0.693]
 [ 0.     0.739 -0.39  -0.548  0.031]
 [ 0.    -0.312  0.434 -0.705  0.466]]


In [6]:
r, c = B.shape
M = np.vstack([np.hstack([np.zeros((c, c)), B.T]), np.hstack([B, np.zeros((r, r))])])
print('M Matrix\n', M)

M Matrix
 [[ 0.     0.     0.     0.     0.    -5.477  0.    -0.    -0.   ]
 [ 0.     0.     0.     0.     0.    23.958 -9.134 -0.    -0.   ]
 [ 0.     0.     0.     0.     0.     0.    -2.562 -1.808  0.   ]
 [ 0.     0.     0.     0.     0.    -0.    -0.    -4.85   3.207]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     4.467]
 [-5.477 23.958  0.    -0.     0.     0.     0.     0.     0.   ]
 [ 0.    -9.134 -2.562 -0.     0.     0.     0.     0.     0.   ]
 [-0.    -0.    -1.808 -4.85   0.     0.     0.     0.     0.   ]
 [-0.    -0.     0.     3.207  4.467  0.     0.     0.     0.   ]]


In [7]:
# use householder reflections to get the permutation matrix and the symmetric tridiagonal matrix
n, m = M.shape
P = np.identity(n)
T = M.copy()
for j in range(n - 2):
    x = T[j + 1:, j]
    H = householderReflection(x, j+1, n)
    T = H @ T @ H
    P = H @ P
print('Permuation Matrix\n', P)
print('Symmetric Tridiagonal Matrix\n', T)


Permuation Matrix
 [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -0.  0.  0.  0.  1. -0.  0.  0.]
 [ 0. -1. -0.  0. -0.  0. -0. -0. -0.]
 [ 0.  0. -0.  0.  0. -0. -1. -0. -0.]
 [ 0. -0.  1.  0. -0.  0. -0. -0.  0.]
 [ 0.  0. -0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0. -0.  1.  0. -0. -0.  0.  0.]
 [ 0.  0.  0.  0. -0.  0.  0. -0. -1.]
 [ 0.  0. -0.  0. -1. -0. -0. -0.  0.]]
Symmetric Tridiagonal Matrix
 [[  0.     -5.477  -0.      0.     -0.      0.      0.      0.      0.   ]
 [ -5.477  -0.    -23.958   0.     -0.      0.      0.      0.      0.   ]
 [ -0.    -23.958  -0.     -9.134  -0.     -0.     -0.     -0.     -0.   ]
 [  0.      0.     -9.134   0.      2.562  -0.     -0.      0.      0.   ]
 [ -0.     -0.     -0.      2.562   0.      1.808  -0.      0.     -0.   ]
 [  0.      0.     -0.     -0.      1.808   0.      4.85   -0.     -0.   ]
 [  0.      0.     -0.      0.     -0.      4.85   -0.     -3.207   0.   ]
 [  0.     -0.     -0.      0.     -0.     -0.     -3.207  -0.      4.467]

In [8]:
def rotation(a, b): # calculate the c and s values in the rotation matrix
    r = np.sqrt(a**2 + b**2)
    c = a/r
    s = -b/r
    return c, s

In [9]:
def givensRotation(T):
    A = T.copy()
    shape = A.shape[0]
    Q = np.identity(shape)
    for i in range(shape-1):
        c, s = rotation(A[i, i], A[i+1, i])
        G = np.identity(shape)
        G[i:i+2, i:i+2] = np.array([[c, -s], [s, c]]) # include the rotation in the identity matrix
        A = G @ A # rotate the current matrix to make the subdiagonal zero
        Q = G @ Q # rotation matrices
    return Q.T, A # transpose Q is Rotation and (Rotation)(Hessian) = R

In [10]:
def qrAlgorithm(T, tolerance=1e-6, iteration=100, sigma=0):
    shape = T.shape[0]
    A_k = T.copy()
    I = np.identity(shape)
    for k in range(iteration):
        # Computes the QR matrix from Hessenberg using Givens Rotation
        Q, R = givensRotation(A_k - sigma*I) # sigma is the initial eigenvalue guess
        A_k = R @ Q + sigma*I
        if np.max(np.abs(np.diag(A_k, k=-1))) < tolerance: # program will terminate when the subdiagonal reached the tolerance
            print('Tolerance reached')
            break
        if k == iteration-1: # program will terminate when max iteration is reached
            print('Max iteration reached')
    return A_k

In [11]:
tolerance = 1e-6
A_k = qrAlgorithm(T, tolerance, sigma=5)
print('Schur Matrix\n', A_k)

Max iteration reached
Schur Matrix
 [[-26.163  -0.      0.      0.      0.      0.      0.      0.     -0.   ]
 [ -0.     26.163   0.     -0.     -0.     -0.     -0.     -0.      0.   ]
 [  0.     -0.     -6.664  -0.      0.      0.     -0.     -0.     -0.   ]
 [ -0.     -0.     -0.     -3.84    0.      0.     -0.      0.      0.   ]
 [ -0.     -0.     -0.      0.     -2.707  -0.      0.     -0.     -0.   ]
 [ -0.      0.     -0.      0.      0.     -0.     -0.      0.      0.   ]
 [  0.     -0.      0.      0.     -0.      0.      2.707  -0.     -0.   ]
 [  0.     -0.     -0.     -0.     -0.     -0.     -0.      6.664   0.   ]
 [ -0.      0.     -0.     -0.     -0.      0.      0.      0.      3.84 ]]


In [12]:
e_value = np.diag(A_k)
print('Calculated Eigenvalues\n', e_value)
print('Numpy Eigenvalues of the M Matrix\n', np.linalg.eig(T)[0])
# get the positive eigenvalues whose absolute value is greater than tolerance then sort it in desc order
s_value = np.flip(np.sort(e_value[np.all([e_value>=0, np.abs(e_value)>tolerance], axis=0)]))
print('Calculates Singular Values\n', s_value)
print('Numpy Singular Values\n', np.linalg.svd(matrix)[1])

Calculated Eigenvalues
 [-26.163  26.163  -6.664  -3.84   -2.707  -0.      2.707   6.664   3.84 ]
Numpy Eigenvalues of the M Matrix
 [ 26.163 -26.163   6.664   3.84    2.707   0.     -6.664  -2.707  -3.84 ]
Calculates Singular Values
 [26.163  6.664  3.84   2.707]
Numpy Singular Values
 [26.163  6.664  3.84   2.707]
